# Evaluate AffectNet March2021
* Author: Sungguk Cha
* eMail: sungguk@ncsoft.com
* Date: 4th Nov. 2022

## Libraries

In [1]:
import copy
import glob
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.metrics import plot_confusion_matrix
import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms
from torchvision.models import resnet101, mobilenet_v2
from tqdm.notebook import tqdm


from robust_optimization import RobustOptimizer

print(f'Torch: {torch.__version__}')
print(f'Timm: {timm.__version__}')

Torch: 1.12.1
Timm: 0.6.11


## Training configurations

In [2]:
affectnet_dir = '../../../../datasets/affectnet'
USE_ENET2=False #False

In [3]:
# Training settings
batch_size = 32 #48# 32# 32 #16 #8 #
epochs = 40
lr = 3e-5
gamma = 0.7
seed = 42
device = 'cuda'
use_cuda = torch.cuda.is_available()
print(use_cuda)

True


In [4]:
IMG_SIZE=260 if USE_ENET2 else 224 # 300 # 80 #
train_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)
print(test_transforms)

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


In [5]:
kwargs = {'num_workers': 0, 'pin_memory': True} if use_cuda else {}

## AffectNet Dataloader

In [6]:
# reference https://github.com/yaoing/DAN/blob/main/affectnet.py
# phase: one of ['train', 'val']
class AffectNet(data.Dataset):
    def __init__(self, aff_path, phase, use_cache=True, transforms=None, force=False):
        self.phase = phase
        self.transforms = transforms
        self.aff_path = aff_path
        self.base_path = os.path.join(self.aff_path, f'{self.phase}_set/')
        
        if use_cache:
            cache_path = os.path.join(aff_path,f'affectnet_{phase}.csv')
            if os.path.exists(cache_path) and not force:
                df = pd.read_csv(cache_path)
            else:
                df = self.get_df()
                df.to_csv(cache_path)
        else:
            df = self.get_df()

        self.data = df[df['phase'] == phase]

        self.file_paths = self.data.loc[:, 'img_path'].values
        self.label = self.data.loc[:, 'label'].values

        self.emotion_labels=['Neutral','Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']
        sample_label, sample_counts = np.unique(self.label, return_counts=True)
        for l, c in zip(sample_label, sample_counts):
            print(f'{self.emotion_labels[l]}: {c} ', end='')
        print(f'\n{len(self)} images')

    def get_df(self):
        base_path = os.path.join(self.aff_path, f'{self.phase}_set/')
        self.base_path = base_path
        data = []
        
        for anno in glob.glob(base_path + 'annotations/*_exp.npy'):
            idx = os.path.basename(anno).split('_')[0]
            img_path = f'images/{idx}.jpg'
            label = int(np.load(anno))
            data.append([self.phase,img_path,label])
        
        return pd.DataFrame(data = data,columns = ['phase','img_path','label'])
    
    def get_weight(self):
        self.emotion_labels=['Neutral','Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']
        self.class_to_idx = {}
        self.idx_to_class = {}
        for i, emotion in enumerate(self.emotion_labels):
            self.class_to_idx[emotion] = i
            self.idx_to_class[i] = emotion
        sample_label, sample_counts = np.unique(self.label, return_counts=True)
        for l, c in zip(sample_label, sample_counts):
            print(f'{self.emotion_labels[l]}: {c} ', end='')
        print('')
        
        cw = 1/sample_counts
        cw /= cw.min()
        class_weights = {i:cwi for i, cwi in zip(sample_label, cw)}
        print(class_weights)
        return class_weights

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        path = os.path.join(self.base_path, self.file_paths[idx])
        image = Image.open(path).convert('RGB')
        label = self.label[idx]

        if self.transforms is not None:
            image = self.transforms(image)
        
        return image, label

In [7]:
trainset = AffectNet(affectnet_dir, 'train', transforms=train_transforms, force=False)
valset = AffectNet(affectnet_dir, 'val', transforms=test_transforms, force=False)
trainloader = data.DataLoader(trainset, batch_size=batch_size, shuffle=True, **kwargs)
valloader = data.DataLoader(valset, batch_size=batch_size, shuffle=False, **kwargs)

Neutral: 74874 Happiness: 134415 Sadness: 25459 Surprise: 14090 Fear: 6378 Disgust: 3803 Anger: 24882 Contempt: 3750 
287651 images
Neutral: 500 Happiness: 500 Sadness: 500 Surprise: 500 Fear: 500 Disgust: 500 Anger: 500 Contempt: 499 
3999 images


In [8]:
class_weights = trainset.get_weight()

Neutral: 74874 Happiness: 134415 Sadness: 25459 Surprise: 14090 Fear: 6378 Disgust: 3803 Anger: 24882 Contempt: 3750 
{0: 1.7952159628175335, 1: 1.0, 2: 5.279665344279037, 3: 9.539744499645138, 4: 21.07478833490122, 5: 35.34446489613463, 6: 5.402097902097902, 7: 35.844}


## Functions

In [9]:
#adapted from https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
def set_parameter_requires_grad(model, requires_grad):
    for param in model.parameters():
        param.requires_grad = requires_grad

In [10]:
# loss function
weights = torch.FloatTensor(list(class_weights.values())).cuda()

def label_smooth(target, n_classes: int, label_smoothing=0.1):
    # convert to one-hot
    batch_size = target.size(0)
    target = torch.unsqueeze(target, 1)
    soft_target = torch.zeros((batch_size, n_classes), device=target.device)
    soft_target.scatter_(1, target, 1)
    # label smoothing
    soft_target = soft_target * (1 - label_smoothing) + label_smoothing / n_classes
    return soft_target

def cross_entropy_loss_with_soft_target(pred, soft_target):
    #logsoftmax = nn.LogSoftmax(dim=-1)
    return torch.mean(torch.sum(- weights*soft_target * torch.nn.functional.log_softmax(pred, -1), 1))

def cross_entropy_with_label_smoothing(pred, target):
    soft_target = label_smooth(target, pred.size(1)) #num_classes) #
    return cross_entropy_loss_with_soft_target(pred, soft_target)

criterion=cross_entropy_with_label_smoothing

In [11]:
models = []
models.append(('EfficientNet_b0_best_afew', '../../models/affectnet_emotions/enet_b0_8_best_afew.pt'))
models.append(('EfficientNet_b0_best_vgaf', '../../models/affectnet_emotions/enet_b0_8_best_vgaf.pt'))

In [12]:
class_to_idx = trainset.class_to_idx
print(class_to_idx)
idx_to_class = trainset.idx_to_class
print(idx_to_class)

{'Neutral': 0, 'Happiness': 1, 'Sadness': 2, 'Surprise': 3, 'Fear': 4, 'Disgust': 5, 'Anger': 6, 'Contempt': 7}
{0: 'Neutral', 1: 'Happiness', 2: 'Sadness', 3: 'Surprise', 4: 'Fear', 5: 'Disgust', 6: 'Anger', 7: 'Contempt'}


In [13]:
pretrained_8 = {0: 'Anger', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happiness', 5: 'Neutral', 6: 'Sadness', 7: 'Surprise'}
new_order_8 = ['Neutral','Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']
new_order_8 = {k: new_order_8.index(v) for k, v in pretrained_8.items()}
print(new_order_8)

{0: 6, 1: 7, 2: 5, 3: 4, 4: 1, 5: 0, 6: 2, 7: 3}


In [14]:
@torch.no_grad()
def eval_pretrained(model, length, dataloader, criterion):
    pretrained_8 = {0: 'Anger', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happiness', 5: 'Neutral', 6: 'Sadness', 7: 'Surprise'}
    new_order_8 = ['Neutral','Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger', 'Contempt']
    new_order_8 = {k: new_order_8.index(v) for k, v in pretrained_8.items()}
    new_order = new_order_8
    model.eval()
    loss = 0.0
    accuracy = 0.0
    for (images, emotions) in tqdm(dataloader):
        images = images.cuda()
        emotions = emotions
        preds = model(images)
        # loss
        # loss += criterion(preds, emotions)
        # accuracy
        preds = torch.argmax(preds, dim=1).cpu()
        preds = preds.apply_(new_order.get)
        acc = torch.eq(preds, emotions).sum()
        accuracy += acc
    loss /= length
    accuracy /= length
    print(f'Accuracy: {accuracy}, Loss: {loss}')

In [15]:
@torch.no_grad()
def eval_pretrained_7(model, length, dataloader, criterion):
    pretrained_8 = {0: 'Anger', 1: 'Disgust', 2: 'Fear', 3: 'Happiness', 4: 'Neutral', 5: 'Sadness', 6: 'Surprise'}
    new_order_8 = ['Neutral','Happiness', 'Sadness', 'Surprise', 'Fear', 'Disgust', 'Anger']
    new_order_8 = {k: new_order_8.index(v) for k, v in pretrained_8.items()}
    new_order = new_order_8
    model.eval()
    loss = 0.0
    accuracy = 0.0
    for (images, emotions) in tqdm(dataloader):
        images = images.cuda()
        emotions = emotions
        preds = model(images)
        # loss
        # loss += criterion(preds, emotions)
        # accuracy
        preds = torch.concat([preds[:, 0:1], preds[:, 2:]], dim=1)
        preds = torch.argmax(preds, dim=1).cpu()
        preds = preds.apply_(new_order.get)
        acc = torch.eq(preds, emotions).sum()
        accuracy += acc
    loss /= length
    accuracy /= (length-499)
    print(f'Accuracy: {accuracy}, Loss: {loss}')

In [16]:
def test(model, valloader):    
    model = torch.load(model)
    model = model.eval().cuda()
    eval_pretrained(model, len(valset), valloader, criterion=None)
    eval_pretrained_7(model, len(valset), valloader, criterion=None)

In [17]:
for name, model in models:
    print(name)
    test(model, valloader)

EfficientNet_b0_best_afew


  0%|          | 0/125 [00:00<?, ?it/s]

Accuracy: 0.5956488847732544, Loss: 0.0


  0%|          | 0/125 [00:00<?, ?it/s]

Accuracy: 0.6394285559654236, Loss: 0.0
EfficientNet_b0_best_vgaf


  0%|          | 0/125 [00:00<?, ?it/s]

Accuracy: 0.6071518063545227, Loss: 0.0


  0%|          | 0/125 [00:00<?, ?it/s]

Accuracy: 0.6422857046127319, Loss: 0.0
